<a href="https://colab.research.google.com/github/NorahAbdulaziz/VideoCaptionModel/blob/main/VideoCaptioningEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Translation-based

Requirements

In [ ]:
!pip install git+https://github.com/Maluuba/nlg-eval.git@master

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from nlgeval.pycocoevalcap.bleu.bleu import Bleu
from nlgeval.pycocoevalcap.rouge.rouge import Rouge
from nlgeval.pycocoevalcap.cider.cider import Cider
from nlgeval.pycocoevalcap.meteor.meteor import Meteor

Source code:
https://github.com/Maluuba/nlg-eval

In [ ]:
hypo_df = pd.read_csv('generatedCaptionsFile', header=None)
hypo_df.columns = ['id', 'text']
print(hypo_df.shape)
hypo_df.head()


In [ ]:
ref_df = pd.read_json('refrenceCaptionsFile')
print(ref_df.shape)
ref_df.head()

In [ ]:
hypo = {}
for index, row in hypo_df.iterrows():
    hypo[row['id']] =  [row['text']]

len(hypo)

In [ ]:
ref = {}
for index, row in ref_df.iterrows():
    ref[row['id']] =  row['caption']

len(ref)

In [ ]:
ref.keys() == hypo.keys()

In [ ]:
def score(ref, hypo):
    scorers = [
      (Bleu(4),["Bleu_1","Bleu_2","Bleu_3","Bleu_4"]),
      (Meteor(),"METEOR"),
      (Rouge(),"ROUGE_L"),
      (Cider(),"CIDEr")
  ]
    final_scores = {}
    for scorer,method in scorers:
        score,scores = scorer.compute_score(ref, hypo)
        if type(score)==list:
            for m,s in zip(method,score):
                final_scores[m] = s
        else:
            final_scores[method] = score

    return final_scores


In [ ]:
# compute score
final_scores = score(ref, hypo)

# print out scores
print('Bleu_1:\t', final_scores['Bleu_1'])
print('Bleu_2:\t', final_scores['Bleu_2'])
print('Bleu_3:\t', final_scores['Bleu_3'])
print('Bleu_4:\t', final_scores['Bleu_4'])
print('METEOR:\t', final_scores['METEOR'])
print('ROUGE_L:', final_scores['ROUGE_L'])
print('CIDEr:\t', final_scores['CIDEr'])

## Video Retrieval-based

Requirements

In [ ]:
! pip3 install nltk

In [ ]:
import nltk
nltk.download('all')
pip install Arabic-Stopwords #from:https://pypi.org/project/Arabic-Stopwords/

In [ ]:
from nltk.corpus import stopwords #english
import arabicstopwords.arabicstopwords as stp #arabic
from nltk.tokenize import word_tokenize
import re
import json, os

Source Code:
https://www.geeksforgeeks.org/python-measure-similarity-between-two-sentences-using-cosine-similarity/

In [ ]:
#English Version
TRAIN_LABEL_PATH = os.path.join('generatedCaptionsFile')
Test_LABEL_PATH = os.path.join('refrenceCaptionsFile')
count=0


test= open(Test_LABEL_PATH) 
data_file= open(TRAIN_LABEL_PATH)

y_data = json.load(test)

for k in data_file:

    X = k.split(',')[1]
    X = re.sub(r'[^\w\s]', '', X)
    for j in y_data:

      for caption in j['caption']:
        #Y = j.split(',')[0]
        Y = re.sub(r'[^\w\s]', '', caption)

        # tokenization
        X_list = word_tokenize(X)
        Y_list = word_tokenize(Y)

        # sw contains the list of stopwords
        sw = stopwords.words('english')
        l1 =[];l2 =[]

        # remove stop words from the string
        X_set = {w for w in X_list if not w in sw}
        Y_set = {w for w in Y_list if not w in sw}

        # form a set containing keywords of both strings
        rvector = X_set.union(Y_set)
      # print(rvector)
        for w in rvector:
          if w in X_set: l1.append(1) # create a vector
          else: l1.append(0)
          if w in Y_set: l2.append(1)
          else: l2.append(0)
      c = 0

      # cosine formula
      for i in range(len(rvector)):
          c+= l1[i]*l2[i]

      cosine = c / float((sum(l1)*sum(l2))**0.5)
      if(cosine > .30  and k.split(',')[0]==j['id']): #threshold equals 30
          count=count+1


In [ ]:
#Arabic Version
TRAIN_LABEL_PATH = os.path.join('generatedCaptionsFile')
Test_LABEL_PATH = os.path.join('refrenceCaptionsFile')
count=0


test= open(Test_LABEL_PATH) 
data_file= open(TRAIN_LABEL_PATH)

y_data = json.load(test)

for k in data_file:

    X = k.split(',')[1]
    X = re.sub(r'[^\w\s]', '', X)
    for j in y_data:

      for caption in j['caption']:
        #Y = j.split(',')[0]
        Y = re.sub(r'[^\w\s]', '', caption)

        # tokenization
        X_list = word_tokenize(X)
        Y_list = word_tokenize(Y)

        l1 =[];l2 =[]

        # remove stop words from the string
        X_set = {w for w in X_list if not stp.is_stop(w) is True}
        Y_set = {w for w in Y_list if not stp.is_stop(w) is True}

        # form a set containing keywords of both strings
        rvector = X_set.union(Y_set)

        for w in rvector:
          if w in X_set: l1.append(1) # create a vector
          else: l1.append(0)
          if w in Y_set: l2.append(1)
          else: l2.append(0)
      c = 0

      # cosine formula
      for i in range(len(rvector)):
          c+= l1[i]*l2[i]

      cosine = c / float((sum(l1)*sum(l2))**0.5)
      if(cosine > .30  and k.split(',')[0]==j['id']): #threshold equals 30
          count=count+1
